In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from decimal import Decimal
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

excel_file_path = "C:/Users/jeffa/OneDrive/Desktop/Smiley Real Estate/ORIGINAL - Smiley Real Estate - Data Analyst Trial - Data Set.xlsx"
sales_df = pd.read_excel(excel_file_path, sheet_name='sales')
calls_df = pd.read_excel(excel_file_path, sheet_name='calls')
agents_df = pd.read_excel(excel_file_path, sheet_name='agents')
customer_df = pd.read_excel(excel_file_path, sheet_name='customers')
markets_df = pd.read_excel(excel_file_path, sheet_name='markets')

In [21]:
df_dict = {'sales_df': sales_df, 'calls_df': calls_df, 'agents_df': agents_df, 'customer_df': customer_df, 'markets_df': markets_df}


for name, df in df_dict.items():
    print(f"\n{name}.describe()")
    print(df.describe())
    print(f"\n{name}.info()")
    print(df.info())



sales_df.describe()
            sale_id   customer_id        amount
count  14172.000000  14172.000000  1.417200e+04
mean    7086.500000  25360.799323  4.880662e+05
std     4091.248342  14327.867008  3.463557e+05
min        1.000000      2.000000  2.230070e+05
25%     3543.750000  13100.750000  4.053662e+05
50%     7086.500000  25529.000000  4.858290e+05
75%    10629.250000  37679.250000  5.655172e+05
max    14172.000000  49989.000000  3.959280e+07

sales_df.info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14172 entries, 0 to 14171
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sale_id      14172 non-null  int64 
 1   customer_id  14172 non-null  int64 
 2   sale_date    14172 non-null  object
 3   amount       14172 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 443.0+ KB
None

calls_df.describe()
             call_id    customer_id
count  175359.000000  175359.000000
mean    87680.000000   249

In [22]:
# Convert date columns into date objects/datatype
sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'])
# calls_df['call_date'] = pd.to_datetime(calls_df['call_date'])
calls_df['call_date'] = pd.to_datetime(calls_df['call_date'], format='%m/%d/%y %H:%M')
customer_df['created_at'] = pd.to_datetime(customer_df['created_at'])

# Convert to numeric, then to Decimal
sales_df['amount'] = pd.to_numeric(sales_df['amount'], errors='coerce')
sales_df['amount'] = sales_df['amount'].apply(lambda x: Decimal(str(x)) if not pd.isna(x) else None)

<span style="font-size:30px; color:blue;">Best Markets</span>

<span style="font-size:20px; color:blue;">A marketing team manager wants to know: What are currently the five best markets for Smiley Real Estate?  What evidence do you have for that?  Also, did you notice anything of interest that we could share with an outside audience (a pure market data trend)?

In [23]:
monthly_amount_stats = sales_df.groupby(sales_df['sale_date'].dt.to_period('M'))['amount'].apply(lambda x: x.astype(float).describe().map('{:,.2f}'.format))
print(monthly_amount_stats)

sale_date       
2025-01    count         1,072.00
           mean        367,178.78
           std          88,195.08
           min         223,007.00
           25%         288,699.00
           50%         361,700.00
           75%         443,942.00
           max         521,940.00
2025-02    count         1,193.00
           mean        400,713.32
           std          85,175.62
           min         246,005.00
           25%         328,859.00
           50%         400,946.00
           75%         475,008.00
           max         545,626.00
2025-03    count         1,399.00
           mean        445,737.08
           std       1,050,966.63
           min         269,361.00
           25%         341,698.50
           50%         417,458.00
           75%         494,013.00
           max      39,592,800.00
2025-04    count         1,340.00
           mean        438,971.88
           std          86,681.19
           min         292,814.00
           25%         363,251.

In [24]:
print(sales_df[sales_df['sale_date'].dt.month == 3].astype({'amount': float}).nlargest(10, 'amount'))
# print()
# print(sales_df[sales_df['sale_date'].dt.month == 3].astype({'amount': float}).nsmallest(10, 'amount'))

      sale_id  customer_id           sale_date      amount
3457     3458        12822 2025-03-25 10:47:00  39592800.0
2947     2948        11051 2025-03-18 17:31:00    568945.0
2333     2334         8712 2025-03-03 13:38:00    568783.0
3083     3084        11538 2025-03-17 13:52:00    568154.0
2594     2595         9757 2025-03-09 15:15:00    568055.0
3232     3233        12018 2025-03-24 10:45:00    567496.0
2395     2396         8993 2025-03-02 15:34:00    567060.0
3065     3066        11476 2025-03-17 09:55:00    567004.0
3679     3680        13642 2025-03-30 17:34:00    566925.0
2843     2844        10676 2025-03-18 14:42:00    566865.0


There is a big outlier in March, a house sold for $39,592,800.00, which is significantly larger than anything else in the sales_df, so either this is an extreme outlier,  or more likely this is an error.  Let's create a new data set and omit this value.

In [25]:
sales_cleaned_df = sales_df
# sales_cleaned_df = sales_cleaned_df.drop(sales_cleaned_df[sales_cleaned_df['amount'] == 39592800].index)   
sales_cleaned_df = sales_cleaned_df.drop(sales_cleaned_df[sales_cleaned_df['amount'] == 39592800].index)

In [26]:
sales_cleaned_df = pd.merge(sales_cleaned_df, customer_df[['customer_id', 'market_id']], on='customer_id', how='left')
print(sales_cleaned_df)

       sale_id  customer_id           sale_date  amount          market_id
0            1            2 2025-01-04 12:45:00  232562         phoenix_az
1            2            3 2025-01-05 14:00:00  473272           tampa_fl
2            3            7 2025-01-13 13:01:00  280980       cleveland_oh
3            4           15 2025-01-12 14:52:00  500904       cleveland_oh
4            5           17 2025-01-08 08:16:00  247848       cleveland_oh
5            6           19 2025-01-04 23:58:00  399207          dallas_tx
6            7           20 2025-01-04 17:19:00  237885         buffalo_ny
7            8           28 2025-01-01 11:54:00  283363    jacksonville_fl
8            9           31 2025-01-08 16:46:00  278547   san_francisco_ca
9           10           33 2025-01-07 13:18:00  288415     san_antonio_tx
10          11           39 2025-01-09 09:16:00  398378       san_diego_ca
11          12           40 2025-01-09 05:34:00  353698         houston_tx
12          13           

In [27]:
market_totals = sales_cleaned_df.groupby(sales_cleaned_df['market_id'])['amount'].sum().reset_index()#.map('{:,.2f}'.format)
print(market_totals.astype({'amount': float}).nlargest(5, 'amount'))
# print(market_totals.nlargest(5, 'amount'))
# print(market_totals[market_totals['market_id']].astype({'amount': float}).nlargest(5, 'amount'))

          market_id       amount
45       seattle_wa  158307881.0
15  grand_rapids_mi  156174194.0
18  jacksonville_fl  150300813.0
24         miami_fl  149932321.0
20     las_vegas_nv  149920376.0


In [28]:
last_30_days = sales_cleaned_df[sales_cleaned_df['sale_date'] >= (pd.to_datetime('11/9/2025') - pd.Timedelta(days=30))]
total_amounts = last_30_days.groupby('market_id')['amount'].sum().astype(float).nlargest(5)
print(total_amounts.map('{:,.2f}'.format))   

market_id
las_vegas_nv         25,867,799.00
grand_rapids_mi      24,744,524.00
miami_fl             24,330,273.00
virginia_beach_va    22,452,628.00
milwaukee_wi         22,417,126.00
Name: amount, dtype: object


<span style="font-size:30px; color:blue;">Best Markets</span>

<span style="font-size:20px; color:blue;">A marketing team manager wants to know: What are currently the five best markets for Smiley Real Estate?  What evidence do you have for that?  Also, did you notice anything of interest that we could share with an outside audience (a pure market data trend)?</span>

<span style="font-size:25px;">Interesting Market Trend to Share</span>

See [Google Sheet tab "Interesting Market Trend to Share w/ Outside"](https://docs.google.com/spreadsheets/d/1V1V6n82gRUbWnah_4vstcnnhxCmBFWHCD-n0GW7jj2U/edit?usp=sharing)

Smiley Real Estate is able to help their customers buy/sell their home on average 8 times faster than the market!  The Smiley Real Estate Team has a median of 5 to 6 days to close depending on the market, whereas the markets median days to sale ranges from 12 to 92!  Depending on the market, Smiley Real Estate days to close is 2 to 15 times faster than the market!  No wonder the Smiley Real Estate team has helped more customers almost every month this year - with an increase of nearly 38% from January to October in 2025!  If you are looking for a real estate team that can get you results fast, give the team at Smiley Real Estate a call.

*There are a number of ways of determining the best market for Smiley Real Estate, but to keep this simple and since the total in sales is what ultimately impacts the bottom line we'll assess the best markets based on which markets had the most dollars in closed sales.  Depending on what time frame is used to define currently, here are the 5 best markets:*

<span style="font-size:25px;">The 5 Best Markets Last 30 Days:</span>

1. las_vegas_nv - $25,867,799.00
2. grand_rapids_mi - $24,744,524.00
3. miami_fl - $24,330,273.00
4. virginia_beach_va - $22,452,628.00
5. milwaukee_wi - $22,417,126.00


<span style="font-size:25px;">The 5 Best Markets for November 2025:</span>

1. las_vegas_nv - $13,231,577
2. miami_fl - $10,744,287
3. richmond_va - $10,606,528
4. jacksonville_fl - $10,353,976
5. washington_dc - $10,348,886


<span style="font-size:25px;">The 5 Best Markets for October 2025:</span>

1. riverside_ca - $23,088,521
2. grand_rapids_mi - $22,488,693
3. seattle_wa - $22,224,664
4. providence_ri - $20,834,350
5. milwaukee_wi - $20,408,234


<span style="font-size:25px;">The 5 Best Markets for 2025:</span>

1. Seattle, WA - $158,307,881
2. Grand Rapids, MI - $156,174,194
3. Jacksonville, FL - $150,300,813
4. Miami, FL - $149,932,321
5. Las Vegas, NV - $149,920,376

<span style="font-size:30px; color:blue;">Best Agents</span>

<span style="font-size:20px; color:blue;">A sales team manager wants to know: Who is the best agent we have in each market?  Why do you think these people are the best - not specifically for each agent, but in general, why are these the top agents?</span>

The best agent can be measured a number of ways, but for the sales manager, they would most likely be interested in:
- total $ in sales
- total number of sales closed
- close rate
- Avg customer review rating
- median number of days to close
    - How does this compare to the market median number of days to close

These would need to be weighted and combined, but to keep this simple, let's look at just two:
1. Close rate in 2024
2. Total dollars closed so far in 2025

<span style="font-size:25px;">Best Agents by 2024 Close Rate</span>

                  market_id                   agent_id         close_rate
1.               atlanta_ga            kwame_patel_238         0.346154
2.                austin_tx           david_hassan_310         0.310811
3.             baltimore_md         arjun_martinez_306         0.238462
4.            birmingham_al            david_nguyen_46         0.262295
5.                boston_ma         arjun_campbell_251         0.261905
6.               buffalo_ny        james_rodriguez_654         0.322404
7.             charlotte_nc          james_ramirez_563         0.210674
8.               chicago_il           emily_walker_196         0.287671
9.            cincinnati_oh           mina_johnson_715         0.383562
10.            cleveland_oh              lucia_clark_4         0.297619
11.             columbus_oh           keisha_scott_367         0.214286
12.               dallas_tx         patricia_young_347         0.356250
13.               denver_co          latoya_walker_552         0.294118
14.              detroit_mi        keisha_mitchell_474         0.327103
15.               fresno_ca             ling_baker_297         0.273543
16.         grand_rapids_mi          isabella_hill_209         0.413793
17.              houston_tx         elizabeth_hill_669         0.219048
18.         indianapolis_in        emily_hernandez_789         0.371429
19.         jacksonville_fl           miguel_clark_518         0.384615
20.          kansas_city_mo            ebony_perez_993         0.289617
21.            las_vegas_nv            tanya_smith_209         0.337748
22.          los_angeles_ca        camila_robinson_734         0.252632
23.           louisville_ky             lucia_king_442         0.352941
24.              memphis_tn          emily_ramirez_702         0.228723
25.                miami_fl           hassan_lewis_806         0.247788
26.            milwaukee_wi          yuki_gonzalez_576         0.316062
27.          minneapolis_mn           charles_hall_449         0.287582
28.            nashville_tn         miguel_johnson_725         0.438776
29.             new_york_ny             jin_harris_955         0.300000
30.        oklahoma_city_ok          jamal_thompson_60         0.340000
31.              orlando_fl         hiroshi_nelson_577         0.224670
32.         philadelphia_pa             emily_khan_248         0.320513
33.              phoenix_az        matthew_ramirez_718         0.311594
34.           pittsburgh_pa        amanda_mitchell_568         0.315476
35.             portland_or             daniel_ali_824         0.269504
36.           providence_ri        zainab_abdullah_560         0.314286
37.              raleigh_nc               raj_hill_659         0.393162
38.             richmond_va         margaret_baker_681         0.309942
39.            riverside_ca            carlos_young_28         0.370000
40.           sacramento_ca           ahmad_walker_939         0.280193
41.       salt_lake_city_ut       linda_washington_623         0.270833
42.          san_antonio_tx          joseph_walker_394         0.333333
43.            san_diego_ca          tyrone_walker_565         0.333333
44.        san_francisco_ca           latoya_scott_543         0.317460
45.             san_jose_ca       richard_martinez_610         0.236559
46.              seattle_wa           william_king_267         0.355140
47.             st_louis_mo         jose_rodriguez_557         0.298780
48.                tampa_fl            minh_harris_725         0.307229
49.       virginia_beach_va        carlos_martinez_859         0.350649
50.           washington_dc         david_williams_651         0.287958



<span style="font-size:25px;">Best Agents by 2025 Total Sales Amount</span>

                  market_id                     agent_id           amount
1.               atlanta_ga           arjun_robinson_452       11110984.0
2.                austin_tx              daniel_white_11       11868074.0
3.             baltimore_md          kenji_hernandez_461        9034126.0
4.            birmingham_al        hassan_washington_575       10490995.0
5.                boston_ma             robert_jones_846        8425685.0
6.               buffalo_ny            imani_thompson_24        8810653.0
7.             charlotte_nc         anthony_williams_805       14075699.0
8.               chicago_il           patricia_patel_670       10590642.0
9.            cincinnati_oh         dorothy_gonzalez_346       12690519.0
10.            cleveland_oh                   ali_hall_3       12971220.0
11.             columbus_oh          latoya_robinson_252       13139450.0
12.               dallas_tx               emily_lewis_40        9976622.0
13.               denver_co            barbara_young_102        9363799.0
14.              detroit_mi          tyrone_thompson_561        9666747.0
15.               fresno_ca           miguel_ramirez_448        7524706.0
16.         grand_rapids_mi        carlos_washington_236        8514920.0
17.              houston_tx               nia_carter_430       10930376.0
18.         indianapolis_in              aya_jackson_584        9303932.0
19.         jacksonville_fl             haruka_smith_116       12388362.0
20.          kansas_city_mo          dorothy_martinez_83       11103548.0
21.            las_vegas_nv             camila_singh_380       12409636.0
22.          los_angeles_ca               priya_young_89       10702076.0
23.           louisville_ky            sarah_ramirez_872        9186275.0
24.              memphis_tn              arjun_patel_669       11411910.0
25.                miami_fl          lakshmi_mohamed_440        8619745.0
26.            milwaukee_wi          dorothy_mohamed_578       12103573.0
27.          minneapolis_mn              jason_davis_983       12270301.0
28.            nashville_tn                 mina_kim_869       10007892.0
29.             new_york_ny           wei_washington_873        9440336.0
30.        oklahoma_city_ok          keisha_campbell_177       11411215.0
31.              orlando_fl          miguel_williams_122       10184267.0
32.         philadelphia_pa            camila_nguyen_189        9028367.0
33.              phoenix_az             daniel_lopez_224       14829419.0
34.           pittsburgh_pa            jessica_evans_748        8870824.0
35.             portland_or             michael_hall_387       13670572.0
36.           providence_ri          zainab_abdullah_560       10479025.0
37.              raleigh_nc             aya_mitchell_864       13495272.0
38.             richmond_va              mary_nguyen_316        9393062.0
39.            riverside_ca            james_jackson_628        9985383.0
40.           sacramento_ca             carlos_jones_151       12213506.0
41.       salt_lake_city_ut               mei_nguyen_346       10958449.0
42.          san_antonio_tx           jason_mitchell_515       12404497.0
43.            san_diego_ca       mohammed_hernandez_381        8094724.0
44.        san_francisco_ca           layla_campbell_987        8593591.0
45.             san_jose_ca              kenji_smith_102        9890752.0
46.              seattle_wa           ahmad_campbell_232       12550246.0
47.             st_louis_mo            joseph_torres_894       11380961.0
48.                tampa_fl                 minh_kim_554        9021676.0
49.       virginia_beach_va                imani_kim_217        9846122.0
50.           washington_dc              david_singh_656       10813423.0

In [29]:
agents_df['close_rate'] = (agents_df['buyer_closes_2024'] + agents_df['seller_closes_2024']) / (agents_df['buyer_customers_sent_2024'] + agents_df['seller_customers_sent_2024'])
agents_df.loc[(agents_df['agent_id'] == 'michael_lewis_922') & (agents_df['market_id'] == 'orlando_fla'), 'market_id'] = 'orlando_fl'   
top_rate_agents=agents_df.loc[agents_df.groupby('market_id')['close_rate'].idxmax()].reset_index()
print(top_rate_agents[['market_id', 'agent_id', 'close_rate']])

            market_id              agent_id  close_rate
0          atlanta_ga       kwame_patel_238    0.346154
1           austin_tx      david_hassan_310    0.310811
2        baltimore_md    arjun_martinez_306    0.238462
3       birmingham_al       david_nguyen_46    0.262295
4           boston_ma    arjun_campbell_251    0.261905
5          buffalo_ny   james_rodriguez_654    0.322404
6        charlotte_nc     james_ramirez_563    0.210674
7          chicago_il      emily_walker_196    0.287671
8       cincinnati_oh      mina_johnson_715    0.383562
9        cleveland_oh         lucia_clark_4    0.297619
10        columbus_oh      keisha_scott_367    0.214286
11          dallas_tx    patricia_young_347    0.356250
12          denver_co     latoya_walker_552    0.294118
13         detroit_mi   keisha_mitchell_474    0.327103
14          fresno_ca        ling_baker_297    0.273543
15    grand_rapids_mi     isabella_hill_209    0.413793
16         houston_tx    elizabeth_hill_669    0

In [30]:
sales_cleaned_df = pd.merge(sales_cleaned_df, customer_df[['customer_id', 'assigned_agent']], on='customer_id', how='left')
top_amount_agents = sales_cleaned_df.groupby('assigned_agent')['amount'].sum().reset_index()#.astype(float).nlargest(5)
top_amount_agents = top_amount_agents.rename(columns={'assigned_agent': 'agent_id'})
top_amount_agents = pd.merge(top_amount_agents, agents_df[['agent_id', 'market_id']], on='agent_id', how='left')
top_amount_agents=top_amount_agents.loc[top_amount_agents.groupby('market_id')['amount'].idxmax()].reset_index()

print(top_amount_agents[['market_id', 'agent_id', 'amount']])

            market_id                agent_id    amount
0          atlanta_ga      arjun_robinson_452  11110984
1           austin_tx         daniel_white_11  11868074
2        baltimore_md     kenji_hernandez_461   9034126
3       birmingham_al   hassan_washington_575  10490995
4           boston_ma        robert_jones_846   8425685
5          buffalo_ny       imani_thompson_24   8810653
6        charlotte_nc    anthony_williams_805  14075699
7          chicago_il      patricia_patel_670  10590642
8       cincinnati_oh    dorothy_gonzalez_346  12690519
9        cleveland_oh              ali_hall_3  12971220
10        columbus_oh     latoya_robinson_252  13139450
11          dallas_tx          emily_lewis_40   9976622
12          denver_co       barbara_young_102   9363799
13         detroit_mi     tyrone_thompson_561   9666747
14          fresno_ca      miguel_ramirez_448   7524706
15    grand_rapids_mi   carlos_washington_236   8514920
16         houston_tx          nia_carter_430  1

<span style="font-size:30px; color:blue;">One area for growth</span>

<span style="font-size:20px; color:blue;">A company executive wants to know: What is one area in which the company might improve their sales or marketing strategy in the future based on what you see in the data?  Make an easy to understand case for that new strategy.</span>

The following is based on the analysis on the [Google Sheets](https://docs.google.com/spreadsheets/d/1V1V6n82gRUbWnah_4vstcnnhxCmBFWHCD-n0GW7jj2U/edit?usp=sharing):
1. markets
2. Agent Performance by Market



Given these stats about Smiley Real Estate:
1. Between 12 - 27 agents in each market
2. Average close rate between 14% - 18%
3. Median days to close between 5 and 6 days for every market - regardless of the market median days to close!
    * calculated as the days between the customer signing up with Smiley Real Estate and the closed sale date.

Since Smiley Real Estate agents have a very fast median days to close, regardless of the median days to close in the market, we should leverage our speed in a slower market.  Smiley Real Estate agents will run circles around the competition and we can market that fact!  We also want a market with lots of inventory and a higher median close price.

The best market for this is Miami, Fl:
1. the slowest sale cycle with a median of 92 days on market
2. the most listings on the market at 47,180
3. A decent median closing price of $560,000

The area for growth that jumps out to me is to focus on recruiting more agents and then marketing in Miami, FL.  Smiley Real Estate agents on average close 15 times faster than the market in Miami, FL!  Recruiting more agents will increase the sales capacity, and if the new agents can be trained up to the Smiley Real Estate standards of the other Miami agents and maintain that median of 6 days to close with a close rate of 16%, Smiley Real Estate will be able to have the biggest impact on their bottom line for their marketing dollars.

<span style="font-size:30px; color:blue;">What if you had more time and data sources?</span>

<span style="font-size:20px; color:blue;"> The data team manager wants to know: Let’s say you had two weeks for this project and any data sets you might imagine.  What are a few ways you could improve your analysis?  Assume you have data engineers in place to help you get the data and set you up for success.</span>

<span style="font-size:25px;">Initial Data Exploration</span>

I would spend more time checking the data for errors.  I did come across and take into account a couple of data errors, like one agent was listed as being in the orlando_fl<span style="color:red;">a</span> market instead of the orlando_fl market like all the other Orlando agents, or the house sold in Detroit, MI for $39 million.  I'm sure there are other data issues I missed.

<span style="font-size:25px;">Improvements for Best Market:</span>

I would want to do some more analysis on how Smiley Real Estate compares to the market.  For example, what % of the listings closed in the last 30 days were ours?  I'd also like to get actual stats for the Total $ Closed in each market for the last 30 days, I just approximated this using the median close price times the number of closed listings.  Then we can see what % of the dollars closed in the last 30 days were closed by Smiley.  Understanding how much of the market share we have can help determine which is our best market and where our opportunities are for growth.

<span style="font-size:25px;">Improvements for Best Agent:</span>

I mentioned previously there were several factors that the sales manager would be interested in, and they would need to be weighted.  I would want to work with the data engineering team and do some machine learning to see how to weigh each of these factors that will result in the largest amount in closed sales so the appropriate incentives can be put in place for the sales team.

I would also want to see the trend in agent performance over time, to see how their close rate goes up or down each year, quarter, and month.


<span style="font-size:25px;">Improvements for Area for Growth</span>

I would want to get more data on how satisfied customers are with their agents.  The number of reviews for each agent ranges from 0 to 15, which is a rate of 0 to 20%, and overall we have 111 reviews from 4,595 customers, a paltry 2.4% that doesn't give us much insight into how happy our agents are leaving our customers.  We want smiley customers and 2% feedback rate doesn't give us enough insight into if we are accomplishing that.  We are missing out on feedback from nearly 98% of our customers.  We should have an incentive in place for customers to give our agents an internal review, and if it is a positive review, ask them to share their experience on a review platform like Google, Yelp, Trust Pilot, etc will strengthen our brand, trust, and SEO.  Also, happy customers are more likely to refer Smiley to others.

<span style="font-size:30px; color:blue;">Describe your process</span>

<span style="font-size:20px; color:blue;">In your own words, describe the process you went through to find the above results.  Mention any tools you used, any significant challenges you faced, and how you overcame them.  No need for perfect language, we’re just interested in your thought process.</span>

I first started exploring the data on the Google Sheet and this Jupyter notebook.  After exploring and getting familiar with the data I began working through the assignment.  For some of the requests I already had created a pivot table or a cell that would answer it, or could quickly and easily make a pivot table or alter a cell. All of the analysis I did could have been done on either tool, but I wanted to use both as a demonstration of my familiarity with both tools.
